In [1]:
%cd ..

c:\Users\habanoz\Desktop\gpt\nb_gpu_trainer


In [2]:
!pip install -e .

Obtaining file:///C:/Users/habanoz/Desktop/gpt/nb_gpu_trainer
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Checking if build backend supports build_editable: started
  Checking if build backend supports build_editable: finished with status 'done'
  Getting requirements to build editable: started
  Getting requirements to build editable: finished with status 'done'
  Preparing editable metadata (pyproject.toml): started
  Preparing editable metadata (pyproject.toml): finished with status 'done'
  Building editable for nb_gpu_trainer (pyproject.toml): started
  Building editable for nb_gpu_trainer (pyproject.toml): finished with status 'done'
  Created wheel for nb_gpu_trainer: filename=nb_gpu_trainer-1.0-0.editable-py3-none-any.whl size=2171 sha256=002f6d83681bb9a96a7b826365d7ce315abb4a8469171a54792201888cde08b9
  Stored in directory: C:\Users\habanoz\AppData\Local\Temp\pip-ephem-wheel-cache-xeef896b\wheels\83\aa\fd\a4d94c85b2ee

## Read confing

In [2]:
%load_ext autoreload
%autoreload 2

from nbtr.model.gpt2 import GPTConfig
from nbtr.trainer import TrainerConfig
from nbtr.tokenizer.tokenizer import Tokenizer
from nbtr.tokenizer.train import Trainer as tok_trainer
from nbtr.hf_backed_trainer import HFBackedTrainer
from nbtr.model.hf_model import HfModel
from nbtr.model.hf_model_config import HfModelConfig

In [3]:
%reload_ext autoreload

In [4]:
from huggingface_hub import login
login()

In [5]:
model_config_file = "config/news_model.yml"
train_config_file = "config/news_trainer.yml"

In [6]:
trainer_config =  TrainerConfig.from_yaml(train_config_file)

In [7]:
trainer_config.out_dir, trainer_config.repo_id, trainer_config.ds_repo_id, trainer_config.data_dir

('haber-90k-gpt',
 'habanoz/haber-90k-gpt-v1.1_test',
 'habanoz/haber-90k-gpt-text',
 'haber-90k-data-test')

## Train Tokenizer

In [15]:
_tok_trainer = tok_trainer(trainer_config.ds_repo_id, trainer_config.repo_id, trainer_config.out_dir)

TypeError: 'Trainer' object is not callable

In [16]:
_tok_trainer.train()

Created repo https://huggingface.co/habanoz/haber-90k-gpt-v1.1_test


tokenizer.model:   0%|          | 0.00/375k [00:00<?, ?B/s]

Files uploaded to the repo at location habanoz/haber-90k-gpt-v1.1_test


## Generate Data

In [8]:
tokenizer = Tokenizer.from_pretrained(trainer_config.repo_id)

tokenizer.model:   0%|          | 0.00/375k [00:00<?, ?B/s]

c:\Users\habanoz\miniconda3\envs\myenv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\habanoz\.cache\huggingface\hub\models--habanoz--haber-90k-gpt-v1.1_test. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [9]:
tokenizer.encode_ds_from_hub(trainer_config.ds_repo_id, trainer_config.data_dir)

Number of 'train' tokens: 62748074
Number of 'validation' tokens: 7015414


## Train

In [11]:
hf_model_config = HfModelConfig(GPTConfig.from_yaml(model_config_file))
hf_model = HfModel.from_pretrained(trainer_config.repo_id)
hf_model.to('cuda')

config.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

c:\Users\habanoz\miniconda3\envs\myenv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\habanoz\.cache\huggingface\hub\models--habanoz--haber-90k-gpt-v1.1_test. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


number of parameters: 13.77M


pytorch_model.bin:   0%|          | 0.00/56.7M [00:00<?, ?B/s]

HfModel(
  (_model): GPT(
    (transformer): ModuleDict(
      (wte): Embedding(8192, 384)
      (wpe): Embedding(1024, 384)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0-5): 6 x Block(
          (ln_1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (attn): CausalSelfAttention(
            (c_attn): Linear(in_features=384, out_features=1152, bias=False)
            (c_proj): Linear(in_features=384, out_features=384, bias=False)
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
          (mlp): MLP(
            (c_fc): Linear(in_features=384, out_features=1536, bias=False)
            (gelu): GELU(approximate='none')
            (c_proj): Linear(in_features=1536, out_features=384, bias=False)
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
      (ln_f): L

In [10]:
hf_model_config.save_pretrained("haber-90k-gpt-v1.1_test", push_to_hub=True)

In [12]:
cfg = TrainerConfig.from_yaml(train_config_file)
trainer = HFBackedTrainer(cfg)

trainer_state.bin:   0%|          | 0.00/113M [00:00<?, ?B/s]

Trainer state is not none
Resume training. Using repo habanoz/haber-90k-gpt-v1.1_test


In [13]:
trainer.train(hf_model.model)

num decayed parameter tensors: 26, with 14,155,776 parameters
num non-decayed parameter tensors: 13, with 4,992 parameters


C:\Users\habanoz\Desktop\gpt\nb_gpu_trainer\src\nbtr\model\gpt2.py:76: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)


Callback iter: 500, best val loss: 4.46705961227417
saving checkpoint to haber-90k-gpt
Eval iter 500: train loss 4.4530, val loss 4.4671, ETA 0:00:00


trainer_state.bin:   0%|          | 0.00/113M [00:00<?, ?B/s]

iter 500: loss 4.6070, run_iter_time 3512.03ms, fb_toks/sec 9330.22, run_fb_toks/sec 9330.22
iter 510: loss 4.4455, run_iter_time 3369.03ms, fb_toks/sec 15738.70, run_fb_toks/sec 9971.07
iter 520: loss 4.5598, run_iter_time 3231.02ms, fb_toks/sec 16474.89, run_fb_toks/sec 10621.45
iter 530: loss 4.4908, run_iter_time 3109.82ms, fb_toks/sec 16229.66, run_fb_toks/sec 11182.27
iter 540: loss 4.4261, run_iter_time 3004.87ms, fb_toks/sec 15904.66, run_fb_toks/sec 11654.51
iter 550: loss 4.4260, run_iter_time 2908.08ms, fb_toks/sec 16086.08, run_fb_toks/sec 12097.67
iter 560: loss 4.4441, run_iter_time 2838.18ms, fb_toks/sec 14833.84, run_fb_toks/sec 12371.29
iter 570: loss 4.3166, run_iter_time 2764.16ms, fb_toks/sec 15618.42, run_fb_toks/sec 12696.00


KeyboardInterrupt: 

In [11]:
trainer.train(hf_model.model)

num decayed parameter tensors: 26, with 14,155,776 parameters
num non-decayed parameter tensors: 13, with 4,992 parameters


C:\Users\habanoz\Desktop\gpt\nb_gpu_trainer\src\nbtr\model\gpt2.py:76: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)


Eval iter 500: train loss 9.0735, val loss 9.0734, ETA 0:00:00
iter 500: loss 9.0764, run_iter_time 3772.57ms, fb_toks/sec 8685.86, run_fb_toks/sec 8685.86
iter 510: loss 7.6443, run_iter_time 3598.12ms, fb_toks/sec 16157.37, run_fb_toks/sec 9433.01
iter 520: loss 7.7173, run_iter_time 3459.50ms, fb_toks/sec 14813.66, run_fb_toks/sec 9971.08
iter 530: loss 7.6600, run_iter_time 3354.84ms, fb_toks/sec 13580.43, run_fb_toks/sec 10332.01
iter 540: loss 7.5307, run_iter_time 3253.46ms, fb_toks/sec 13997.37, run_fb_toks/sec 10698.55


In [12]:
trainer.train(hf_model.model)

num decayed parameter tensors: 26, with 14,155,776 parameters
num non-decayed parameter tensors: 13, with 4,992 parameters


C:\Users\habanoz\Desktop\gpt\nb_gpu_trainer\src\nbtr\model\gpt2.py:76: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)


Eval iter 500: train loss 9.0808, val loss 9.0817, ETA 0:00:00
iter 500: loss 9.0793, run_iter_time 3864.02ms, fb_toks/sec 8480.29, run_fb_toks/sec 8480.29
iter 510: loss 7.6431, run_iter_time 3673.96ms, fb_toks/sec 16688.89, run_fb_toks/sec 9301.15
iter 520: loss 7.7132, run_iter_time 3506.57ms, fb_toks/sec 16383.57, run_fb_toks/sec 10009.39


KeyboardInterrupt: 

In [10]:
trainer.train(hf_model.model)

num decayed parameter tensors: 26, with 14,155,776 parameters
num non-decayed parameter tensors: 13, with 4,992 parameters


C:\Users\habanoz\Desktop\gpt\nb_gpu_trainer\src\nbtr\model\gpt2.py:76: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)


Eval iter 500: train loss 9.0589, val loss 9.0590, ETA 0:00:00
iter 500: loss 9.0638, run_iter_time 3804.09ms, fb_toks/sec 8613.88, run_fb_toks/sec 8613.88
iter 510: loss 7.6491, run_iter_time 3651.38ms, fb_toks/sec 14390.79, run_fb_toks/sec 9191.57
iter 520: loss 7.7009, run_iter_time 3503.95ms, fb_toks/sec 15051.81, run_fb_toks/sec 9777.60
iter 530: loss 7.6508, run_iter_time 3376.33ms, fb_toks/sec 14708.65, run_fb_toks/sec 10270.70
iter 540: loss 7.5347, run_iter_time 3249.80ms, fb_toks/sec 15522.68, run_fb_toks/sec 10795.90
iter 550: loss 7.4411, run_iter_time 3158.02ms, fb_toks/sec 14051.15, run_fb_toks/sec 11121.42
iter 560: loss 7.3552, run_iter_time 3093.02ms, fb_toks/sec 13065.27, run_fb_toks/sec 11315.81
iter 570: loss 7.2336, run_iter_time 3029.52ms, fb_toks/sec 13331.29, run_fb_toks/sec 11517.36
iter 580: loss 7.1971, run_iter_time 2978.37ms, fb_toks/sec 13013.41, run_fb_toks/sec 11666.96
iter 590: loss 7.0704, run_iter_time 2895.34ms, fb_toks/sec 15254.76, run_fb_toks/sec 

KeyboardInterrupt: 

: 

In [9]:
trainer.train(hf_model.model)

num decayed parameter tensors: 26, with 14,155,776 parameters
num non-decayed parameter tensors: 13, with 4,992 parameters


C:\Users\habanoz\Desktop\gpt\nb_gpu_trainer\src\nbtr\model\gpt2.py:76: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  y = torch.nn.functional.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)


Callback iter: 0, best val loss: 9.085393905639648
Eval iter 0: train loss 9.0863, val loss 9.0854, ETA 0:00:00
iter 0: loss 9.0898, run_iter_time 1644.97ms, fb_toks/sec 4980.02, run_fb_toks/sec 4980.02
iter 10: loss 8.5029, run_iter_time 1660.27ms, fb_toks/sec 4556.24, run_fb_toks/sec 4937.65
iter 20: loss 8.1967, run_iter_time 1663.55ms, fb_toks/sec 4838.71, run_fb_toks/sec 4927.75
iter 30: loss 7.7021, run_iter_time 1677.89ms, fb_toks/sec 4533.46, run_fb_toks/sec 4888.32
iter 40: loss 7.3702, run_iter_time 1691.20ms, fb_toks/sec 4523.48, run_fb_toks/sec 4851.84
iter 50: loss 6.8880, run_iter_time 1714.89ms, fb_toks/sec 4248.86, run_fb_toks/sec 4791.54
iter 60: loss 6.9289, run_iter_time 1734.61ms, fb_toks/sec 4284.16, run_fb_toks/sec 4740.80
iter 70: loss 6.7574, run_iter_time 1777.75ms, fb_toks/sec 3782.06, run_fb_toks/sec 4644.93
iter 80: loss 6.4745, run_iter_time 1788.78ms, fb_toks/sec 4339.05, run_fb_toks/sec 4614.34
iter 90: loss 6.3318, run_iter_time 1797.94ms, fb_toks/sec 43

TypeError: HfModel.save() missing 1 required positional argument: 'trainer_config'